In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from fase.seal import * 
import numpy as np
import time, sys 
from sklearn.metrics import r2_score
from read_data import *
from server import *
from user import *

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [5]:
class Key:
    def __init__(self, context):
        keygen = KeyGenerator(context)
        self.public_key = keygen.create_public_key()
        self.private_key = keygen.secret_key() 
        self.relin_keys = keygen.create_relin_keys()
        self.gal_keys = keygen.create_galois_keys()

In [6]:
#CKKS Parameters
params = EncryptionParameters(scheme_type.ckks)
poly_modulus_degree = 2**15
# poly_modulus_degree = 2**14
params.set_poly_modulus_degree(poly_modulus_degree)
params.set_coeff_modulus(CoeffModulus.Create(poly_modulus_degree, 
                                             [60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]))
# params.set_coeff_modulus(CoeffModulus.Create(poly_modulus_degree, [59, 40, 40, 40, 40, 40, 40, 40, 40, 59]))
# params.set_coeff_modulus(CoeffModulus.Create(poly_modulus_degree, [60, 40, 40, 40, 40, 40, 40, 40, 60]))

context = SEALContext(params)
keygen = KeyGenerator(context)
keys = Key(context)

## What's wrong with keys.private_key??
keys.private_key = keygen.secret_key()
keys.public_key = keygen.create_public_key()
keys.relin_keys = keygen.create_relin_keys()
keys.gal_keys = keygen.create_galois_keys()

user = USER(keys, context)

In [16]:
dim = 4
imgs = 200

##Datasets

# X_ori = wine_data()
# X_ori = wine_data("white")
# X_ori = air_quality_data()
# X_ori = parkinson_data()
X_ori = load_mnist("training", path='./MNIST/', no_of_imgs=imgs)
# X_ori = load_fashion_mnist("training", no_of_imgs=imgs)
# size = X_ori.shape
# # X_ori = X_ori.reshape((size[0], size[1]*size[2]))

#X_ori = load_yale()

Reading images


In [17]:
mu = np.mean(X_ori, axis=0)
X = X_ori - mu
print("Shape of data =", X.shape)

X_cipher = user.encrypt_data(X)
print("No.of levels =", context.get_context_data(X_cipher[0].parms_id()).chain_index())
# print(X_cipher.shape, user.incr, user.vec_len)

Shape of data = (200, 256)
N = 200 k = 256 j = 64
Encrypting
No.of levels = 11


In [18]:
server = SERVER(keys, user.incr, dim, context)
start = time.time()
eig_val, eig_vec = server.power_iteration(X_cipher)
end = time.time()
vec = user.extract_eigen_vectors(eig_vec, X.shape[1])
with open("eigen_vector.npy", "wb") as file:
    np.save(file, vec)

X_red = X.dot(vec)
print(X_red.shape)
X_new = X_red.dot(vec.T) + mu 
print("No.of re-encryptions =", server.count)
print("Time taken =",(end-start)/60,"mins")
print("R2 score =", r2_score(X_ori, X_new))

Eigen vector : 1
final eig_val 4.3481411264411076e-05
Eigen vector : 2
final eig_val 4.2520857978660557e-05
Eigen vector : 3
final eig_val 4.237642143347425e-05
Eigen vector : 4
final eig_val 4.526333093992197e-05
(200, 4)
No.of re-encryptions = 112
Time taken = 4.154178134600321 mins
R2 score = 4.119968255444917e-17


In [16]:
server = SERVER(keys, user.incr, dim, context)
start = time.time()
eig_val, eig_vec = server.power_iteration(X_cipher)
end = time.time()
vec = user.extract_eigen_vectors(eig_vec, X.shape[1])
with open("eigen_vector.npy", "wb") as file:
    np.save(file, vec)

X_red = X.dot(vec)
print(X_red.shape)
X_new = X_red.dot(vec.T) + mu 
print("No.of re-encryptions =", server.count)
print("Time taken =",(end-start)/60,"mins")
print("R2 score =", r2_score(X_ori, X_new))

Eigen vector : 1
final eig_val 1.5834979512077e-05
Eigen vector : 2
final eig_val 1.5610710051855865e-05
Eigen vector : 3
final eig_val 1.429437611646756e-05
Eigen vector : 4
final eig_val 1.6556844923344145e-05
(165, 4)
No.of re-encryptions = 112
Time taken = 3.5043052554130556 mins
R2 score = 4.723100816700354e-12


In [18]:
vec.shape

(256, 4)

In [20]:
X_new.shape

(165, 256)